In [12]:
import json
import os

def read_data(datafile=os.path.join(os.getcwd(), '.data', 'master.json')):
    with open(datafile, mode='r') as f:
        lines = [json.loads(l) for l in f.readlines()]
        return lines

data = read_data()

print(data[:10])

[{'feed_name': 'The Economist - Science and Technology', 'img_loc': None, 'pub_datetime': '2018-02-08 15:45:55', 'summary': ' \xa0  fructose is the sweetest of the natural sugars. as its name suggests, it is found mainly in fruits. its job seems to be to appeal to the sweet tooths of the vertebrates these fruit have evolved to be eaten by, the better to scatter their seeds far and wide. fructose is also, however, often added by manufacturers of food and drink, to sweeten their products and make them appeal to one species of vertebrate in particular, namely  homo sapiens . and that may be a problem, because too much fructose in the diet seems to be associated with liver disease and type 2 diabetes.  the nature of this association has been debated for years. some argue that the effect is indirect. they suggest that, because sweet tastes suppress the feeling of being full (the reason why desserts, which come at the end of a meal, are sweet), consuming foods rich in fructose encourages ove

In [13]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
from collections import Counter
init_notebook_mode(connected=True)

kwds = []
[kwds.extend(d['kwds']) for d in data]

data = [go.Histogram(x=kwds)]
plot(data)




'file:///home/cmustard/Projects/spyndicated/spyndicated/temp-plot.html'